In [25]:
import requests
import time
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service


In [26]:
#Defini una url base y la pagina que contiene los modelos 

BASE_URL = "https://www.ford.pe"
MODELOS_URL = "https://www.ford.pe/todos/"

In [27]:
def init_driver():
    """
    Inicializa el driver de Selenium para Firefox utilizando un perfil existente.
    """
    options = Options()
    options.headless = True  # Modo headless para evitar abrir ventanas visibles

    driver = webdriver.Firefox(options=options)
    return driver

In [28]:
def obtener_categorias(driver):
    """
    Accede a la página principal de modelos y extrae las categorías principales: SUV's, Pick-ups y Autos.
    """
    print("Accediendo a la página de modelos:", MODELOS_URL)
    driver.get(MODELOS_URL)
    time.sleep(10)  # Espera para que la página cargue completamente

    # Buscar las categorías por encabezados
    categorias = driver.find_elements(By.XPATH, "//div[@class='splitter section']")
    
    if not categorias:
        print("No se encontraron categorías en la página.")
        return []

    modelos = []  # Lista para almacenar los enlaces a las páginas de modelos

    for categoria_div in categorias:
        try:
            # Encuentra el nombre de la categoría (por ejemplo, SUV's)
            categoria_titulo = categoria_div.find_element(By.XPATH, ".//h2").text
            print(f"Categoría encontrada: {categoria_titulo}")

            # Encuentra los enlaces "Conoce más" dentro de la categoría
            enlaces_modelos = categoria_div.find_elements(By.XPATH, ".//a[contains(text(), 'Conoce más')]")
            for enlace in enlaces_modelos:
                url_modelo = enlace.get_attribute("href")
                modelos.append({"categoria": categoria_titulo, "url": url_modelo})
        except Exception as e:
            print(f"Error al procesar una categoría: {e}")

    return modelos


In [29]:
def extraer_ficha_tecnica(driver, url_modelo):
    """
    Navega a la página del modelo y extrae el enlace de la ficha técnica (PDF).
    """
    print(f"Accediendo a la página del modelo: {url_modelo}")
    driver.get(url_modelo)
    time.sleep(10)  # Esperar para que la página cargue completamente

    try:
        # Buscar el enlace al PDF de la ficha técnica
        ficha_element = driver.find_element(By.XPATH, "//a[contains(text(), 'Ficha técnica')]")
        ficha_url = ficha_element.get_attribute("href")
        print(f"Ficha técnica encontrada: {ficha_url}")
        return ficha_url
    except Exception as e:
        print(f"No se encontró ficha técnica en {url_modelo}: {e}")
        return None

In [30]:
def main():
    driver = init_driver()

    # Paso 1: Obtener los modelos desde la página principal
    modelos = obtener_categorias(driver)
    if not modelos:
        print("No se encontraron modelos.")
        driver.quit()
        return

    # Paso 2: Extraer las fichas técnicas de cada modelo
    resultados = []
    for modelo in modelos:
        ficha_tecnica = extraer_ficha_tecnica(driver, modelo["url"])
        resultados.append({
            "categoria": modelo["categoria"],
            "url_modelo": modelo["url"],
            "ficha_tecnica": ficha_tecnica
        })

    driver.quit()


if __name__ == "__main__":
    main()

Accediendo a la página de modelos: https://www.ford.pe/todos/
Error al procesar una categoría: Message: Unable to locate element: .//h2; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Error al procesar una categoría: Message: Unable to locate element: .//h2; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/conten